# Library and api key import

# What does this function do


In [ ]:
# This function will this function retrieves all
#the comments inside the first x videos that appear on youtube, for a particular string. It will include comment
# replies for each comment. Likely delivering a bigger quantity of results that getcomments_for_multiple_videos_no_replies
# but with  a cost, it will increase a LOT execution time. We recommend executing this for a small number of videos
# maxing 10. 

videolist = []
totallist = []
commentthreadlist = []

# THis are three key elements, used for us to store the results of our code , we created them on the outside of the functions
# because the processes that will happen on this code can be really extensive, to the point where we separated the
# code in three parts. TO make sure the variables all always there , we created the variables globally. 
#The first part declares a function that obtains comment responses for a particular comment thread
# the second part will create a function that obtains comments for a particular video id. The third part gets the list
# of videos for the string we give, and executed the function for comment gathering in a loop, delivering all of them in
# a dataframe. To understand the code inside , we recommend reading the 'individual youtube api functions' , there 
# we use very similar functions independently and explain the code. 

# Library and api key import

In [ ]:
#Establishing api key, and calling the libraries to be used
import json

with open('youtube_credentials.json') as cred_data:
    info = json.load(cred_data)
    api_key = info["youtube_apikeyjson"]


# Here we import library, and create a youtube object that will represent our conection to the api, through which we
# will perform our requests
from apiclient.discovery import build
youtube = build('youtube' , 'v3' , developerKey= api_key)

import pandas as pd


# THis are two key elements, used for us to store the results of our code



# Get comment responses 

In [5]:
def getcommentresponses(commentid, objectpart = "snippet" , resultnumber = 100 , textFormat = "plainText"  ):
    callobject = youtube.commentThreads().list(part = objectpart , maxResults = resultnumber, 
                                               textFormat = "plainText" , id = commentid)
    results = callobject.execute()
    
    for item in results["items"]:
            
        comment = item["snippet"]["topLevelComment"]
        author = comment["snippet"]["authorDisplayName"]
        text = comment["snippet"]["textDisplay"]
        resultdic = {"user" : author ,"usercomment" : comment["snippet"]["textDisplay"], 
                   "likecount" : comment["snippet"]["likeCount"], "commentid" : item["id"], 
                     "publishingdate" : comment["snippet"]["publishedAt"]}
                
        totallist.append(resultdic)
    

# Get comment threads

In [6]:

# This is the function used to retrieve comments for a particular video id.  We separate it so the get comments for a keyword code
# is not so long, and it can be easily modified afterwards.

def commentsearch(videoIdentification, partofcomment = "snippet", numbrofresult = 100):
    callobject = youtube.commentThreads().list(part = partofcomment, videoId = videoIdentification, maxResults = numbrofresult, 
                                               textFormat = "plainText")

    results = callobject.execute()
    
    
    # here we are creating a function for extracting all the interesting information of a comment, we will be calling
    # it  a lot later. The functions first creates a number of variables with the key elements extracted from
    # each interaction of the result object, afterwards, creates a dictionary with those results and adds it to the
    #totallist, a list with all comments for the keyword, all of them, from all the videos.  This function
    # also creates a list of the comments ids, that will be vital for afterwards, extracting the replies if we want them
    
    def incosearch() :
        for item in results["items"]:
            comment = item["snippet"]["topLevelComment"]
            author = comment["snippet"]["authorDisplayName"]
            text = comment["snippet"]["textDisplay"]
            resultdic = {"user" : author ,"usercomment" : comment["snippet"]["textDisplay"], 
                   "likecount" : comment["snippet"]["likeCount"], "commentid" : item["id"] ,
                     "publishingdate" : comment["snippet"]["publishedAt"] }
            totallist.append(resultdic)
            commentthreadlist.append(resultdic["commentid"])
            
    incosearch() # this is a first call, without it, the first page will get skipped. 

        
    if ("nextPageToken" in results) == True : 
        while ("nextPageToken" in results) == True :
            callobject = youtube.commentThreads().list(part = partofcomment, videoId = videoIdentification, maxResults = numbrofresult, 
                                               textFormat = "plainText", pageToken = results["nextPageToken"])
            results = callobject.execute()
            
            incosearch() # this will cll incosearch()
            
      
    
    elif ("nextPageToken" in results) == False :
        pass
    
    print("everything ok thus far")
    print(str(len(commentthreadlist)) + " are the number of comment threads, replies not included")
    
    # Here we get the comment replies , we added to this function in this one, because without it. There would be 
    # No correct execution on the last call 
    
    for i2 in commentthreadlist:
            getcommentresponses(i2)
    print(str(len(totallist)) + " are the number of comments extracted on this video, replies included.")
    


   




# Get comments for multiple videos, for a particular string with replies 

In [7]:

def getcmultividswithR(stringquery, numresults = 25 ,
              beforepublished = None , afterpublished = None , rcode = None):
    videolist = []
    totallist = []
    
    # Here we call the youtube library, and create a youtube object that represents our conection with the api. 
    from apiclient.discovery import build
    
    youtube = build('youtube' , 'v3' , developerKey= api_key)
    
    
    #this function gets a list with videoids for a query, the stringquery we give. If we specify an rcode argument
    # or a date before or after publish, this will affect the list of videos obtained. The maximum numbers of videos we 
    # can obtain are 50. 
    
    funcall = youtube.search().list( q = stringquery ,part = "id", type = 'video', maxResults = numresults , 
                             publishedBefore = beforepublished , publishedAfter = afterpublished, regionCode = rcode ) 
    result = funcall.execute()
    
    # after this,collect all the comments from the videos with a loop, using the commentsearchfunction that we have
    # created earlier in this document. 
    

    for videoitem in range(0,len(result["items"])):
        videoitemid = result["items"][videoitem]["id"]["videoId"]
        videolist.append(videoitemid)
        
    
    for i in videolist:
        commentthreadlist = []
        commentsearch(videoIdentification = i )
        
    
    finalr = pd.DataFrame(totallist)
    
            
    print(len(totallist))
    return(finalr)

    
    
getcmultividswithR("potato", 5)

everything ok thus far
3794 are the number of comment threads, replies not included
7588 are the number of comments extracted on this video, replies included.
0


""
